In [1]:
import pandas as pd
import requests
import json
import time

##### translate

In [6]:
import deepl

In [ ]:
df = pd.read_excel('/content/drive/MyDrive/polit_model/Wahlomat_Total.xlsx', index_col=0)

In [ ]:
df = df.sample(3, random_state = 42)

In [ ]:
import deepl

auth_key = "d43f89b0-bf84-198e-f2ac-ade89584c635:fx"  # Replace with your key
translator = deepl.Translator(auth_key)

In [ ]:
for i in df.index:
  df.loc[i,'text_EN'] = translator.translate_text(df.loc[i,'text'],source_lang='DE', target_lang="EN-US")

In [ ]:
df.loc[:,['text','text_EN']]

In [ ]:
df.text_EN.astype(str).to_list()[0]

In [12]:
import transformers
transformers.__version__

'4.3.3'

In [10]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B")


KeyError: 'gptj'

In [ ]:
model = AutoModelForCausalLM.from_pretrained("EleutherAI/gpt-j-6B")

In [ ]:
prompt = f"""[Original]: Algeria recalled its ambassador to Paris on Saturday and closed its airspace to French military planes a day later after the French president made comments about the northern Africa country. 
        [Paraphrase]: Last Saturday, the Algerian government recalled its ambassador and stopped accepting French military airplanes in its airspace. It happened one day after the French president made comments about Algeria.
        ###
        [Original]: President Macron was quoted as saying the former French colony was ruled by a "political-military system" with an official history that was based not on truth, but on hatred of France.
        [Paraphrase]: Emmanuel Macron said that the former colony was lying and angry at France. He also said that the country was ruled by a "political-military system".
        ###
        [Original]: The diplomatic spat came days after France cut the number of visas it issues for citizens of Algeria and other North African countries.
        [Paraphrase]: Diplomatic issues started appearing when France decided to stop granting visas to Algerian people and other North African people.
        ###
        [Original]: {df.text_EN.astype(str).to_list()[0]}
        [Paraphrase]:"""

In [ ]:
input_ids = tokenizer(prompt, return_tensors="pt").input_ids
gen_tokens = model.generate(input_ids, do_sample=True, temperature=0.9, max_length=100,)
gen_text = tokenizer.batch_decode(gen_tokens)[0]

In [ ]:
gen_text

In [ ]:
import nlpcloud
client = nlpcloud.Client("gpt-j", "1d277fe0c958ce5092b107d8ac51736107223657", gpu=True)
generation = client.generation("""[Original]: Algeria recalled its ambassador to Paris on Saturday and closed its airspace to French military planes a day later after the French president made comments about the northern Africa country. 
        [Paraphrase]: Last Saturday, the Algerian government recalled its ambassador and stopped accepting French military airplanes in its airspace. It happened one day after the French president made comments about Algeria.
        ###
        [Original]: President Macron was quoted as saying the former French colony was ruled by a "political-military system" with an official history that was based not on truth, but on hatred of France.
        [Paraphrase]: Emmanuel Macron said that the former colony was lying and angry at France. He also said that the country was ruled by a "political-military system".
        ###
        [Original]: The diplomatic spat came days after France cut the number of visas it issues for citizens of Algeria and other North African countries.
        [Paraphrase]: Diplomatic issues started appearing when France decided to stop granting visas to Algerian people and other North African people.
        ###
        [Original]: After a war lasting 20 years, following the decision taken first by President Trump and then by President Biden to withdraw American troops, Kabul, the capital of Afghanistan, fell within a few hours to the Taliban, without resistance.
        [Paraphrase]:""",
    length_no_input=True,
    end_sequence="\n###",
    remove_end_sequence=True,
    remove_input=True,
    min_length=0,
    max_length=50)
print(generation["generated_text"])